In [2]:
import os
import time
import random
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urlparse

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

In [46]:
# 로컬 경로 설정 (본인 환경에 맞게 수정)
base_path = "D:/김동영/11_Github/mygit-1/DAT/25-1프로젝트/"
handle_path = os.path.join(base_path, "소셜핸들.xlsx")
result_csv_path = os.path.join(base_path, "exolyt_result.csv")

# 엑셀에서 Username 추출
handle_df = pd.read_excel(handle_path)
usernames = handle_df.iloc[:, 0].dropna().apply(lambda x: x.lstrip('@')).tolist()

print("✅ 사용자 수:", len(usernames))
print("🎯 미리보기:", usernames[:])

✅ 사용자 수: 305
🎯 미리보기: ['theonlyhaven', 'turquoise.dec', 'ttalgi.ellie', 'srsaskin_', '_anjello', 'jf.skin24', 'cinthia_dosamples', 'lee.ly1012', 'prettywrinkles', 'xevelynluu', 'chloelouiseliveslife', 'melissahemmingsley', 'kbootyjenny', 'skincare.ninja', 'ugc_withjessi', 'so.calmama', 'jessicaroseugc', 'kayskwa', 'jennasiee', 'ugcxmichelle', 'autumnsaro', 'tinaangelinaaa', 'crystalschob3', 'life.w.lex4', 'jensnomz', 'srsdigitaldiary', 'eloise_ugc', 'shellllls_lifeeee', 'xokena_', 'mindfulugc', 'nooni.ugc', 'heyitsaliquinn', 'glo.ssary', 'ugc.karla', 'pretty.squirrelss', 'danielaapryl', 'allbriwoo', 'grw.alii_', 'jilliangottlieb', '_taniagrover', 'zhala.beauty', 'skinbykristin', 'lifewtamia', 'sunscreen.addict', 'toninlanaa', 'ft.angela', 'pinaydora', 'yangmediary', 'd_dudley4', 'beauty__bymegs', 'witandarte', 'beautycloudlove', 'psdiwan22', 'auraofmahmuda', 'vickyoffishal', '__allaboutskincare__', 'klymi.na', 'jeracoolio', 'razan.alani', 'angelinajarboe', 'waves_for_dayzz', 'jmeans77',

In [47]:
import json

def apply_login_cookies(driver, cookie_path):
    with open(cookie_path, 'r', encoding='utf-8') as f:
        json_data = json.load(f)
    
    cookies_raw = json_data.get('cookies', [])
    
    # 문자열인 쿠키 → 객체로 변환
    cookies = []
    for c in cookies_raw:
        if isinstance(c, str):
            try:
                cookies.append(json.loads(c))
            except:
                continue
        else:
            cookies.append(c)

    # 첫 방문 (쿠키 설정을 위해)
    driver.get("https://www.exolyt.com")
    time.sleep(2)

    for cookie in cookies:
        try:
            cookie.pop('domain', None)  # 도메인 충돌 방지
            driver.add_cookie(cookie)
        except Exception as e:
            print(f"⚠️ 쿠키 추가 실패: {cookie.get('name', 'N/A')} → {e}")


In [48]:
def handle_track_account_popup(driver):
    try:
        # 1️⃣ 첫 번째 팝업: "Track account" 버튼 클릭
        track_button = WebDriverWait(driver, 3).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/main/div[2]/div[2]/div/div/div/button'))
        )
        track_button.click()
        print("✅ 1단계: 'Track account' 클릭")

        # 2️⃣ 두 번째 팝업: "Great!" 버튼 클릭 (뜨면)
        try:
            great_button = WebDriverWait(driver, 3).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/main/div[4]/div/div/div/button'))
            )
            great_button.click()
            print("✅ 2단계: 'Great!' 팝업 닫기 완료")
        except:
            print("ℹ️ 'Great!' 팝업 없음")

        # 3️⃣ 다시 한번 'Track account' 클릭 (Analytics 탭 열리게 하기 위해)
        time.sleep(1)
        try:
            track_button = WebDriverWait(driver, 3).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/main/div[2]/div[2]/div/div/div/button'))
            )
            track_button.click()
            print("✅ 3단계: 'Track account' 두 번째 클릭 완료")
        except:
            print("ℹ️ 3단계에서 Track 버튼 없음 (이미 닫힘 가능성)")
    except:
        print("ℹ️ 'Account data is outdated' 팝업 없음")


In [ ]:
def crawl_exolyt(username):
    try:
        options = webdriver.ChromeOptions()
        # options.add_argument("--headless")  # 필요 시 주석 해제
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("start-maximized")
        options.add_argument("user-agent=Mozilla/5.0")

        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

        # ✅ 1. 로그인 쿠키 적용
        cookie_path = os.path.join(base_path, "exolyt_login_state.json")
        driver.get("https://www.exolyt.com")
        time.sleep(2)
        apply_login_cookies(driver, cookie_path)

        # ✅ 2. 유저 페이지 이동
        driver.get(f"https://exolyt.com/user/tiktok/{username}")
        time.sleep(random.uniform(4, 6))
        handle_track_account_popup(driver)

        # ✅ 3. 로딩 완료 대기
        WebDriverWait(driver, 10).until(
            EC.invisibility_of_element_located((By.CLASS_NAME, "ds-fade-in"))
        )

        # ✅ 4. Overview 클릭
        overview_btn = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Overview')]"))
        )
        driver.execute_script("arguments[0].click();", overview_btn)
        print("✅ Overview 클릭 완료")
        time.sleep(2)

        # ✅ 5. 'Choose date range' 체크박스 클릭
        checkbox_btn = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//input[@type='checkbox' and contains(@class, 'cursor-pointer')]"))
        )
        driver.execute_script("arguments[0].click();", checkbox_btn)
        print("✅ 'Choose date range' 체크박스 클릭 완료")
        time.sleep(2)
        
        # ✅ 6. 'Choose date range' 체크박스 클릭 (이미 체크되었으면 스킵)
        checkbox_btn = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//input[@type='checkbox' and contains(@class, 'cursor-pointer')]"))
        )
        is_checked = driver.execute_script("return arguments[0].checked;", checkbox_btn)
        if not is_checked:
            driver.execute_script("arguments[0].click();", checkbox_btn)
            print("✅ 'Choose date range' 체크박스 클릭 완료")
        else:
            print("ℹ️ 'Choose date range' 체크박스는 이미 선택되어 있음")
        time.sleep(2)

        # ✅ 6. HTML 파싱
        soup = BeautifulSoup(driver.page_source, "html.parser")

        # ✅ 7. Account Influence 수치 추출
        def extract_value(label):
            tag = soup.find("div", string=lambda t: t and label.lower() in t.lower())
            if tag:
                value_tag = tag.find_next_sibling("div")
                return value_tag.get_text(strip=True) if value_tag else None
            return None

        data = {
            "username": username,
            "total_views": extract_value("Total views"),
            "likes": extract_value("Likes"),
            "total_comments": extract_value("Total comments"),
            "total_video_shares": extract_value("Total video shares"),
            "total_engagements": extract_value("Total engagements")
        }

        # ✅ 8. Median 수치 추가 추출
        def extract_median_value(label):
            tag = soup.find("div", string=lambda t: t and label.lower() in t.lower())
            if tag:
                value_tag = tag.find_next_sibling("div")
                return value_tag.get_text(strip=True) if value_tag else None
            return None

        data.update({
            "median_views": extract_median_value("Median views"),
            "median_likes": extract_median_value("Median likes"),
            "median_comments": extract_median_value("Median comments"),
            "median_video_shares": extract_median_value("Median video shares")
        })

        print("📊 최종 수집 데이터:", data)
        return data

    except Exception as e:
        print(f"❌ {username} - 실패: {e}")
        return None

    finally:
        try:
            driver.quit()
        except:
            pass


In [50]:
results = []

for i, username in enumerate(usernames):
    print(f"\n[{i+1}/{len(usernames)}] ▶ {username}")
    result = crawl_exolyt(username)
    if result:
        results.append(result)
        print(f"✅ {username} - 완료")
    else:
        print(f"❌ {username} - 저장되지 않음")


[1/305] ▶ theonlyhaven
ℹ️ 'Account data is outdated' 팝업 없음
✅ Overview 클릭 완료
✅ 'Choose date range' 체크박스 클릭 완료
📊 최종 수집 데이터: {'username': 'theonlyhaven', 'total_views': '491.6M', 'likes': '60.0M', 'total_comments': '260.4K', 'total_video_shares': '952.7K', 'total_engagements': None, 'median_views': '36.4K', 'median_likes': '2.7K', 'median_comments': '15', 'median_video_shares': '25'}
✅ theonlyhaven - 완료

[2/305] ▶ turquoise.dec
ℹ️ 'Account data is outdated' 팝업 없음
✅ Overview 클릭 완료
✅ 'Choose date range' 체크박스 클릭 완료
📊 최종 수집 데이터: {'username': 'turquoise.dec', 'total_views': '149.1M', 'likes': '33.0M', 'total_comments': '61.6K', 'total_video_shares': '301.7K', 'total_engagements': None, 'median_views': '310.8K', 'median_likes': '36.2K', 'median_comments': '90', 'median_video_shares': '450'}
✅ turquoise.dec - 완료

[3/305] ▶ ttalgi.ellie
ℹ️ 'Account data is outdated' 팝업 없음
✅ Overview 클릭 완료
✅ 'Choose date range' 체크박스 클릭 완료
📊 최종 수집 데이터: {'username': 'ttalgi.ellie', 'total_views': '368.6K', 'likes':

In [51]:
result_df = pd.DataFrame(results)
result_df.to_csv(result_csv_path, index=False, encoding='utf-8-sig')
print(f"\n📁 저장 완료: {result_csv_path}")


📁 저장 완료: D:/김동영/11_Github/mygit-1/DAT/25-1프로젝트/exolyt_result.csv


In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('D:/김동영/11_Github/mygit-1/DAT/25-1프로젝트/exolyt_result.csv')

df_total_views_1 = df[df['total_views'] == '1']
df_total_views_1

,username,total_views,likes,total_comments,total_video_shares,total_engagements,median_views,median_likes,median_comments,median_video_shares
74,upasanaarora,1,42,NaN,NaN,NaN,NaN,NaN,NaN,NaN
148,heyitsclarissaa,1,472.3K,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149,nakeziaa,1,3.3M,NaN,NaN,NaN,NaN,NaN,NaN,NaN
192,renlauskin,1,9.3K,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193,pixelheartsbeauty,1,770,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
274,timeforrice,1,4.2K,NaN,NaN,NaN,NaN,NaN,NaN,NaN
275,jhamheartsthis,1,163.8K,NaN,NaN,NaN,NaN,NaN,NaN,NaN
276,aglimpseofglow,1,2.6K,NaN,NaN,NaN,NaN,NaN,NaN,NaN
277,mariana.ugc,1,4.2K,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# 로컬 경로 설정 (본인 환경에 맞게 수정)
base_path = "D:/김동영/11_Github/mygit-1/DAT/25-1프로젝트/"
# handle_path = os.path.join(base_path, "소셜핸들.xlsx")
result_csv_path = os.path.join(base_path, "exolyt_result2.csv")

# 엑셀에서 Username 추출
# handle_df = pd.read_excel(handle_path)
usernames = df_total_views_1.iloc[:, 0].dropna().apply(lambda x: x.lstrip('@')).tolist()

print("✅ 사용자 수:", len(usernames))
print("🎯 미리보기:", usernames[:])

✅ 사용자 수: 90
🎯 미리보기: ['upasanaarora', 'heyitsclarissaa', 'nakeziaa', 'renlauskin', 'pixelheartsbeauty', 'jelloskinsecrets', 'ugc_adriiana', 'skin_likee', 'glowological', 'luv4ikeuuu', 'fabulous.mira.skincare', 'ugcale', 'skinbyhelen', 'ellasolves', 'skinwithe1', 'ugcwithmeriem', 'joselinastyle', 'starfaceorganics', 'ishmadiaries', 'stanskincare', 'itshazeldiaries', 'skinwithclods', 'greyyychoi', 'heartz4pitii', 'mattrandon', 'melon_park', '_lilyis', 'keytoglo', 'abbyrosemaureen', 'kiki.dee__', 'denitsab3auty', 'lilac__05', 'dreamyglowyskin', 'bethyful_kbeauty', 'maddysbeautytalk', 'lovemisomeskin', 'ugcwithkpm', 'betweendots', 'chen.ugc', 'aera.ugc', 'skincarewithxia_', 'skinhippie27', 'klea.sotiri', 'zahraxbeauty', 'katysayfi', 'sonuugrg_', 'anika_kozlova', 'itstessjohnson', 'looksbyhiba', 'itsmelissaga', 'zoe.aesthetic.grw', 'thedikshadiary', 'akshayareddykontham', 'lifestylewithfatima', 'bosslady__carla', 'thehydratedhunny', 'grace_forevermore_', 'brendasbeautyblog_', 'curateandglow'

In [6]:
import json

def apply_login_cookies(driver, cookie_path):
    with open(cookie_path, 'r', encoding='utf-8') as f:
        json_data = json.load(f)
    
    cookies_raw = json_data.get('cookies', [])
    
    # 문자열인 쿠키 → 객체로 변환
    cookies = []
    for c in cookies_raw:
        if isinstance(c, str):
            try:
                cookies.append(json.loads(c))
            except:
                continue
        else:
            cookies.append(c)

    # 첫 방문 (쿠키 설정을 위해)
    driver.get("https://www.exolyt.com")
    time.sleep(2)

    for cookie in cookies:
        try:
            cookie.pop('domain', None)  # 도메인 충돌 방지
            driver.add_cookie(cookie)
        except Exception as e:
            print(f"⚠️ 쿠키 추가 실패: {cookie.get('name', 'N/A')} → {e}")


In [7]:
def handle_track_account_popup(driver):
    try:
        # 1️⃣ 첫 번째 팝업: "Track account" 버튼 클릭
        track_button = WebDriverWait(driver, 3).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/main/div[2]/div[2]/div/div/div/button'))
        )
        track_button.click()
        print("✅ 1단계: 'Track account' 클릭")

        # 2️⃣ 두 번째 팝업: "Great!" 버튼 클릭 (뜨면)
        try:
            great_button = WebDriverWait(driver, 3).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/main/div[4]/div/div/div/button'))
            )
            great_button.click()
            print("✅ 2단계: 'Great!' 팝업 닫기 완료")
        except:
            print("ℹ️ 'Great!' 팝업 없음")

        # 3️⃣ 다시 한번 'Track account' 클릭 (Analytics 탭 열리게 하기 위해)
        time.sleep(1)
        try:
            track_button = WebDriverWait(driver, 3).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/main/div[2]/div[2]/div/div/div/button'))
            )
            track_button.click()
            print("✅ 3단계: 'Track account' 두 번째 클릭 완료")
        except:
            print("ℹ️ 3단계에서 Track 버튼 없음 (이미 닫힘 가능성)")
    except:
        print("ℹ️ 'Account data is outdated' 팝업 없음")


In [ ]:
def crawl_exolyt(username):
    try:
        # ✅ 0. 크롬 설정
        options = webdriver.ChromeOptions()
        # options.add_argument("--headless")  # 필요 시 주석 해제
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("start-maximized")
        options.add_argument("user-agent=Mozilla/5.0")

        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

        # ✅ 1. 로그인 쿠키 적용
        cookie_path = os.path.join(base_path, "exolyt_login_state.json")
        driver.get("https://www.exolyt.com")
        time.sleep(2)
        apply_login_cookies(driver, cookie_path)

        # ✅ 2. 유저 페이지 이동
        driver.get(f"https://exolyt.com/user/tiktok/{username}")
        time.sleep(random.uniform(4, 6))
        handle_track_account_popup(driver)

        # ✅ 3. 로딩 완료 대기
        WebDriverWait(driver, 10).until(
            EC.invisibility_of_element_located((By.CLASS_NAME, "ds-fade-in"))
        )

        # ✅ 4. 팝업 닫기 (있으면 닫고, 없으면 통과)
        try:
            close_btn = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/main/div[3]/div/svg'))
            )
            driver.execute_script("arguments[0].click();", close_btn)
            print("✅ 팝업 닫기 완료")
            time.sleep(2)
        except Exception:
            print("ℹ️ 팝업 없음 → 그대로 진행")

        # ✅ 5. Overview 클릭
        overview_btn = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Overview')]"))
        )
        driver.execute_script("arguments[0].click();", overview_btn)
        print("✅ Overview 클릭 완료")
        time.sleep(2)

        # ✅ 6. 'Choose date range' 체크박스 클릭 (이미 체크되었으면 스킵)
        checkbox_btn = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//input[@type='checkbox' and contains(@class, 'cursor-pointer')]"))
        )
        is_checked = driver.execute_script("return arguments[0].checked;", checkbox_btn)
        if not is_checked:
            driver.execute_script("arguments[0].click();", checkbox_btn)
            print("✅ 'Choose date range' 체크박스 클릭 완료")
        else:
            print("ℹ️ 'Choose date range' 체크박스는 이미 선택되어 있음")
        time.sleep(2)

        # ✅ 7. HTML 파싱
        soup = BeautifulSoup(driver.page_source, "html.parser")

## 주석 처리한 부분 수정
        # # ✅ 7. Account Influence 수치 추출
        # def extract_value(label):
        #     tag = soup.find("div", string=lambda t: t and label.lower() in t.lower())
        #     if tag:
        #         value_tag = tag.find_next_sibling("div")
        #         return value_tag.get_text(strip=True) if value_tag else None
        #     return None

        # data = {
        #     "username": username,
        #     "total_views": extract_value("Total views"),
        #     "likes": extract_value("Likes"),
        #     "total_comments": extract_value("Total comments"),
        #     "total_video_shares": extract_value("Total video shares"),
        #     "total_engagements": extract_value("Total engagements")
        # }

        # # ✅ 8. Median 수치 추가 추출
        # def extract_median_value(label):
        #     tag = soup.find("div", string=lambda t: t and label.lower() in t.lower())
        #     if tag:
        #         value_tag = tag.find_next_sibling("div")
        #         return value_tag.get_text(strip=True) if value_tag else None
        #     return None

        # data.update({
        #     "median_views": extract_median_value("Median views"),
        #     "median_likes": extract_median_value("Median likes"),
        #     "median_comments": extract_median_value("Median comments"),
        #     "median_video_shares": extract_median_value("Median video shares")
        # })

        # print("📊 최종 수집 데이터:", data)
        # return data

    except Exception as e:
        print(f"❌ {username} - 실패: {e}")
        return None

    finally:
        try:
            driver.quit()
        except:
            pass

In [9]:
results = []

for i, username in enumerate(usernames):
    print(f"\n[{i+1}/{len(usernames)}] ▶ {username}")
    result = crawl_exolyt(username)
    if result:
        results.append(result)
        print(f"✅ {username} - 완료")
    else:
        print(f"❌ {username} - 저장되지 않음")


[1/90] ▶ upasanaarora
ℹ️ 'Account data is outdated' 팝업 없음
ℹ️ 팝업 없음 → 그대로 진행
✅ Overview 클릭 완료
✅ 'Choose date range' 체크박스 클릭 완료
📊 최종 수집 데이터: {'username': 'upasanaarora', 'total_views': '1', 'likes': '42', 'total_comments': 'N/A', 'total_video_shares': 'N/A', 'total_engagements': None, 'median_views': '', 'median_likes': '', 'median_comments': '', 'median_video_shares': ''}
✅ upasanaarora - 완료

[2/90] ▶ heyitsclarissaa
ℹ️ 'Account data is outdated' 팝업 없음
ℹ️ 팝업 없음 → 그대로 진행
✅ Overview 클릭 완료
✅ 'Choose date range' 체크박스 클릭 완료
📊 최종 수집 데이터: {'username': 'heyitsclarissaa', 'total_views': '1', 'likes': '472.3K', 'total_comments': 'N/A', 'total_video_shares': 'N/A', 'total_engagements': None, 'median_views': '', 'median_likes': '', 'median_comments': '', 'median_video_shares': ''}
✅ heyitsclarissaa - 완료

[3/90] ▶ nakeziaa
ℹ️ 'Account data is outdated' 팝업 없음
ℹ️ 팝업 없음 → 그대로 진행
✅ Overview 클릭 완료
✅ 'Choose date range' 체크박스 클릭 완료
📊 최종 수집 데이터: {'username': 'nakeziaa', 'total_views': '1', 'likes': '3.3M',

KeyboardInterrupt: 

In [2]:
# 로컬 경로 설정 (본인 환경에 맞게 수정)
base_path = "D:/김동영/11_Github/mygit-1/DAT/25-1프로젝트/"
result_csv_path = os.path.join(base_path, "exolyt_result2.csv")

In [ ]:
result_df = pd.DataFrame(results)
result_df.to_csv(result_csv_path, index=False, encoding='utf-8-sig')
print(f"\n📁 저장 완료: {result_csv_path}")